In [1]:
import pandas as pd

# Load Train Dataset
train_df = pd.read_csv('..\indexing\data\splitted_data\\v2\\train.csv')
train_df['status'] = train_df['status'].apply(lambda x : str(x).strip())

#Load Validation Dataset
val_df = pd.read_csv('..\indexing\data\splitted_data\\v2\\val.csv')

In [2]:
#instansi : the name of the institution from the reference table
#reference : input from the user which will later be predicted
#status : actual label, "yes" if equal (True), "no" if different (False)

train_df.head()

,instansi,status,reference
0,lembaga pembiayaan ekspor indonesia,yes,lembaga pembiayaan ekspor
1,lembaga pembiayaan ekspor indonesia,yes,lpei
2,lembaga pembiayaan ekspor indonesia,yes,lembaga ekspor indonesia
3,lembaga pembiayaan ekspor indonesia,yes,pembiayaan ekspor indonesia
4,lembaga pembiayaan ekspor indonesia,no,pembiayaan indonesia


In [ ]:
#List of Pretrained Model (for more model visit : https://huggingface.co/indobenchmark)
list_model = ['indobert-base-p1', 'indobert-base-p2']
INDEX = 0

In [ ]:
#Import Library
import torch
import numpy as np
from transformers import BertTokenizer

#Load tokenizer for Indobert
tokenizer = BertTokenizer.from_pretrained(f'indobenchmark/{list_model[INDEX]}')

#labels mapping
labels = {'no':0,
          'yes':1,
          }

In [ ]:
#Create Dataset Class to feed into model
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['status']]
        self.texts = [tokenizer(df['instansi'][i], df['reference'][i],
                               padding='max_length', max_length = 512, truncation=True, 
                               return_tensors="pt") for i in range(df.index[0],df.index[-1] + 1)]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
#Model Building
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()
    
        self.bert = BertModel.from_pretrained('indobenchmark/{}'.format(list_model[INDEX]))
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [3]:
#Train function
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    #Load train and validation dataloader which will be the input model (tensor)
    train, val = Dataset(train_data), Dataset(val_data)
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    #use GPU if available, if not use cpu 
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    #Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    #use GPU if available and send model into GPU
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    #List for tracking train loss, val loss, train acc and val acc per epoch
    list_epoch = []
    list_train_loss = []
    list_val_loss = []
    train_progress = pd.DataFrame()
    best_acc_val = 0

    #training iteration
    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            #training
            for train_input, train_label in tqdm(train_dataloader):
                
                #label
                train_label = train_label.to(device)

                #input model (mask and input_id)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                #model predict (train data)
                output = model(input_id, mask)
                
                #train loss and acc calculation
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            #validation
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            #Save best model
            val_acc_temp = total_acc_val / len(val_data)
            if val_acc_temp > best_acc_val :
                torch.save(model, "Models/Best_Model/{}".format(list_model[INDEX])) 
            
            #save training progress
            best_acc_val = val_acc_temp
            list_epoch.append(epoch_num+1)
            list_train_loss.append(total_loss_train / len(train_data))
            list_val_loss.append(total_loss_val / len(val_data))

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

    #create training progress dataframe, for reporting
    train_progress['epoch'] = list_epoch
    train_progress['train_loss'] = list_train_loss
    train_progress['val_loss'] = list_val_loss

    #save training_progress into csv
    train_progress.to_csv('result/training_progress/training_progress_{}.csv'.format(list_model[INDEX]))

In [ ]:
#Parameter
EPOCHS = 1
model = BertClassifier()

#learning rate
LR = 1e-6

#call train function
train(model, train_df, val_df, LR, EPOCHS)

In [ ]:
#Evaluate model with test data 
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    #print testing accuracy
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [ ]:
#Load test dataset
test_df = pd.read_csv('..\indexing\data\splitted_data\\v2\\test.csv') 

#Load best model
model = torch.load("Models/Best_Model/{}".format(list_model[INDEX]))  

#Evaluate
evaluate(model, test_df)